<center><h1>Progresso da Vacinação contra a COVID-19</h1></center>
<br>
<center><img src="https://www.gov.br/mcti/pt-br/acompanhe-o-mcti/noticias/2022/07/boletim-de-vigilancia-genomica-do-sars-cov-2-aponta-mudancas-no-perfil-epidemiologico-do-virus/viruscovid.jpg" width=600></img></center>  

<br>

<h1 style='background:#483D8B; border:0; color:white'><center>Introdução</center></h1> 

O dataset possui as seguintes informações: 

* **País** - país para o qual as informações de vacinação são fornecidas;     
* **Código ISO do País** - código ISO para o país;   
* **Data** - data para a entrada de dados; para algumas datas temos apenas as vacinações diárias, para outras, apenas o total (cumulativo);   
* **Número total de vacinações** - número absoluto de imunizações no país;  
* **Número total de pessoas vacinadas** - uma pessoa, dependendo do esquema de imunização, receberá uma ou mais (normalmente 2) vacinas; em um determinado momento, o número de vacinações pode ser maior do que o número de pessoas;  
* **Número total de pessoas totalmente vacinadas** - número de pessoas que receberam o conjunto completo de imunização de acordo com o esquema de imunização (normalmente 2); em um determinado momento, pode haver um certo número de pessoas que receberam uma vacina e outro número (menor) de pessoas que receberam todas as vacinas do esquema;  
* **Vacinações diárias (brutas)** - para uma determinada entrada de dados, o número de vacinações para essa data/país;  
* **Vacinações diárias** - para uma determinada entrada de dados, o número de vacinações para essa data/país;  
* **Total de vacinações por cem** - razão (em percentual) entre o número de vacinações e a população total até a data no país;  
* **Número total de pessoas vacinadas por cem** - razão (em percentual) entre a população imunizada e a população total até a data no país;  
* **Número total de pessoas totalmente vacinadas por cem** - razão (em percentual) entre a população totalmente imunizada e a população total até a data no país;   
* **Número de vacinações por dia** - número de vacinações diárias para aquele dia e país;   
* **Vacinações diárias por milhão** - razão (em ppm) entre o número de vacinações e a população total para a data atual no país;    
* **Vacinas utilizadas no país** - número total de vacinas utilizadas no país (até a data);    
* **Nome da fonte** - fonte da informação (autoridade nacional, organização internacional, organização local etc.);   
* **Website da fonte** - website da fonte de informação;    

<a id="0"></a>

### Conteúdo  

* <a href='#1'>Preparação da análise</a>  
* <a href='#2'>Quais vacinas são usadas em cada país?</a>    
* <a href='#3'>Qual esquema de vacinação é mais utilizado?</a>    
* <a href='#4'>Quantos estão vacinados (total e como percentual da população)?</a>    
* <a href='#5'>Como a vacinação evoluiu</a>  

### Última atualização


In [104]:
import datetime
import os
import time

# Caminho para o diretório local
directory_path = r"C:\Users\Aluno Manhã\Documents\Covid19"

# Verificar se o diretório existe
if os.path.exists(directory_path):
    try:
        with os.scandir(directory_path) as dir_entries:
            for entry in dir_entries:
                # Obter o timestamp da última modificação
                unix_timestamp = int(entry.stat().st_mtime)
                utc_time = time.gmtime(unix_timestamp)
                print(f"Dataset last time updated: {utc_time.tm_year}-{utc_time.tm_mon}-{utc_time.tm_mday}")
                break
    except Exception as e:
        print(f"An error occurred while accessing the directory: {e}")
else:
    print(f"Directory not found: {directory_path}")

# Obter a data e hora atuais para o notebook
ldt = datetime.datetime.now()
print(f"Notebook last time updated: {ldt.year}-{ldt.month}-{ldt.day}")

Dataset last time updated: 2024-9-3
Notebook last time updated: 2024-9-3


<a id="1"></a><h1 style='background:#483D8B; border:0; color:white'><center>Preparação da Análise</center></h1>

Inicializamos os pacotes Python que usaremos para ingestão, preparação e visualização dos dados. Utilizaremos principalmente o Plotly para visualização.
Em seguida, lemos o arquivo de dados e agregamos as informações em alguns campos (país, código ISO e vacinas - ou seja, o esquema de vacinação usado em um determinado país).

Vamos focar principalmente em:
* Quais esquemas de vacinação são utilizados em vários países;  
* Número total de vacinações e percentual de vacinações;  
* Vacinações diárias e vacinações diárias por milhão;
* Total de pessoas vacinadas e percentual de pessoas vacinadas;   

O dataset também contém informações sobre o número total de pessoas completamente vacinadas (e seu percentual).

Visualizamos os valores mais recentes (máximos) e também a variação ao longo do tempo dos valores mencionados acima.

In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import warnings

# Inicializar o modo do notebook para Plotly
init_notebook_mode(connected=True)

# Ignorar avisos
warnings.filterwarnings("ignore")


In [106]:
import pandas as pd

# Caminho atualizado para o arquivo CSV
data_df = pd.read_csv("C:\\Users\\Aluno Manhã\\Documents\\Covid19\\country_vaccinations.csv")

# Exibir as primeiras linhas do DataFrame para verificar se a leitura foi bem-sucedida
print(data_df.head())


       country iso_code        date  total_vaccinations  people_vaccinated  \
0  Afghanistan      AFG  2021-02-22                 0.0                0.0   
1  Afghanistan      AFG  2021-02-23                 NaN                NaN   
2  Afghanistan      AFG  2021-02-24                 NaN                NaN   
3  Afghanistan      AFG  2021-02-25                 NaN                NaN   
4  Afghanistan      AFG  2021-02-26                 NaN                NaN   

   people_fully_vaccinated  daily_vaccinations_raw  daily_vaccinations  \
0                      NaN                     NaN                 NaN   
1                      NaN                     NaN              1367.0   
2                      NaN                     NaN              1367.0   
3                      NaN                     NaN              1367.0   
4                      NaN                     NaN              1367.0   

   total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
0                 

In [107]:
import pandas as pd

# Ler o arquivo CSV
data_df = pd.read_csv("C:\\Users\\Aluno Manhã\\Documents\\Covid19\\country_vaccinations.csv")

# Agrupar os dados e calcular o máximo para as colunas selecionadas
country_vaccine = data_df.groupby(["country", "iso_code", "vaccines"])[[
    'total_vaccinations',
    'total_vaccinations_per_hundred',
    'daily_vaccinations',
    'daily_vaccinations_per_million',
    'people_vaccinated',
    'people_vaccinated_per_hundred',
    'people_fully_vaccinated',
    'people_fully_vaccinated_per_hundred'
]].max().reset_index()

# Renomear as colunas
country_vaccine.columns = [
    "Country", "iso_code", "Vaccines", "Total vaccinations", "Percent", 
    "Daily vaccinations", "Daily vaccinations per million", 
    "People vaccinated", "People vaccinated per hundred",
    "People fully vaccinated", "People fully vaccinated percent"
]

# Exibir as primeiras linhas do DataFrame para verificar o resultado
print(country_vaccine.head())


       Country iso_code                                           Vaccines  \
0  Afghanistan      AFG    BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech   
1      Albania      ALB  Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...   
2      Algeria      DZA                      Oxford/AstraZeneca, Sputnik V   
3      Andorra      AND                Oxford/AstraZeneca, Pfizer/BioNTech   
4       Angola      AGO                                 Oxford/AstraZeneca   

   Total vaccinations  Percent  Daily vaccinations  \
0            662003.0     1.70             13921.0   
1            880912.0    30.61             17565.0   
2           2500000.0     5.70             22664.0   
3             51107.0    66.15              1376.0   
4           1380174.0     4.20             33700.0   

   Daily vaccinations per million  People vaccinated  \
0                           358.0           484737.0   
1                          6104.0           515884.0   
2                           517.0     

<small><a href='#0'>Go to top</a></small>  


<a id="2"></a><h1 style='background:#483D8B; border:0; color:White'><center>Quais vacinas são utilizadas em cada país?</center></h1>

Pressione o botão **Output** para visualizar cada país e as vacinas utilizadas em cada um

In [108]:
vaccines = country_vaccine.Vaccines.unique()
for v in vaccines:
    countries = country_vaccine.loc[country_vaccine.Vaccines==v, 'Country'].values
    print(f"Vacinas: {v}: \nPaises: {list(countries)}\n")

Vacinas: BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech: 
Paises: ['Afghanistan']

Vacinas: Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V: 
Paises: ['Albania', 'Bosnia and Herzegovina', 'Philippines', 'Tunisia']

Vacinas: Oxford/AstraZeneca, Sputnik V: 
Paises: ['Algeria', 'Ghana', 'Guyana', 'Kenya', 'Nicaragua']

Vacinas: Oxford/AstraZeneca, Pfizer/BioNTech: 
Paises: ['Andorra', 'Australia', 'Cape Verde', 'Cayman Islands', 'Costa Rica', 'Isle of Man', 'Oman', 'Panama', 'Saudi Arabia', 'Slovenia', 'Sweden']

Vacinas: Oxford/AstraZeneca: 
Paises: ['Angola', 'Anguilla', 'Antigua and Barbuda', 'Bahamas', 'Bangladesh', 'Barbados', 'Bhutan', 'Botswana', 'British Virgin Islands', 'Burkina Faso', 'Cook Islands', "Cote d'Ivoire", 'Democratic Republic of Congo', 'Dominica', 'Eswatini', 'Ethiopia', 'Falkland Islands', 'Fiji', 'French Polynesia', 'Georgia', 'Grenada', 'Jamaica', 'Kosovo', 'Lesotho', 'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Montserrat', 'Myanmar', 'Nauru', 'Nigeria',

In [109]:
fig = px.choropleth(locations=country_vaccine['Country'], 
                    locationmode="country names",
                    color=country_vaccine['Vaccines'],
                    title="Paises usando cada vacina (cores diferentes para cada vacina)",
                    height = 800
                   )
fig.update_layout({'legend_orientation':'v'})
fig.update_layout({'legend_title':'Esquema de vacinas:'})
fig.show()

<small><a href='#0'>Ir ao topo</a></small>  

<a id="3"></a><h1 style='background:#483D8B; border:0; color:white'><center>Qual o esquema de vacinação mais utilizado?</center></h1>

## Visão Geral

Vamos primeiro analisar o esquema de vacinação utilizado de forma geral (não dividido por países).

In [110]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot

# Agrupar os dados e calcular o máximo para as colunas selecionadas
vaccine = data_df.groupby(["vaccines"])[[
    'total_vaccinations',
    'total_vaccinations_per_hundred',
    'daily_vaccinations',
    'daily_vaccinations_per_million'
]].max().reset_index()

# Renomear as colunas
vaccine.columns = [
    "Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
    "Daily vaccinations per million"
]

# Função para desenhar o gráfico de barras
def draw_trace_bar_vaccine(data, feature, title, xlab, ylab, color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
        x=data['Vaccines'],
        y=data[feature],
        marker=dict(color=color),
        text=data['Vaccines']
    )
    data = [trace]

    layout = dict(
        title=title,
        xaxis=dict(
            title=xlab, showticklabels=True, tickangle=45,
            zeroline=True, zerolinewidth=1, zerolinecolor='grey',
            showline=True, linewidth=2, linecolor='black', mirror=True,
            tickfont=dict(size=10, color='black')
        ), 
        yaxis=dict(
            title=ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
            showline=True, linewidth=2, linecolor='black', mirror=True
        ),
        plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor='rgba(0, 0, 0, 0)',
        hovermode='closest',
        height=800
    )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='draw_trace')

# Exemplo de chamada da função (substitua 'Total vaccinations' pelo nome da coluna desejada para o gráfico)
draw_trace_bar_vaccine(vaccine, 'Total vaccinations', 'Total de Vacinações organizados por tipo de vacina:', 'Tipo de Vacina', 'Total Vaccinations')

In [111]:
draw_trace_bar_vaccine(vaccine, 'Total vaccinations', 'Total por esquema de vacinação', 'Vacina', 'Vacinação total', "darkmagenta" )

O mapeamento é o seguinte:

* Moderna, Pfizer/BioNTech - EUA;
* CNBG, Sinovac - China;
* Oxford/AstraZeneca, Pfizer/BioNTech - Reino Unido;
* Pfizer/BioNTech - principalmente na UE;
* Pfizer/BioNTech, Sinopharm - UAE;
* Sinovac - Turquia;
* Covaxin, Covishield - Índia;


<small><a href='#0'>Ir para o Início</a></small>  

## Por países
Para visualizar a distribuição do esquema de vacinação por países, utilizaremos representações em treemap.

Analisaremos os valores de vacinação total, vacinação diária, bem como o total de pessoas vacinadas.

<font color="red">Nota</font>: clique em um item do treemap para navegar para baixo na estrutura da árvore e expandir o ramo atual.

In [112]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'Total vaccinations',
                title="Vacinações totais por país, agrupadas por esquema de vacinação")
fig.show()

In [113]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'Total vaccinations',
                title="Vacinação total por paises, agrupado por esquema de vacinação:")
fig.show()

In [114]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'Daily vaccinations',
                title="Vacinação Diária por país agrupado por esquema de vacinação:")
fig.show()

In [115]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'People vaccinated',
                title="Pessoas vacinadas por país agrupadas por esquema de vacina:")
fig.show()

<small><a href='#0'>Ir para o Início</a></small>  

<a id="4"></a><h1 style='background:#483D8B; border:0; color:white'><center>Quantos estão vacinados (total como porcentagem da população)?</center></h1>

Vamos agora analisar as estatísticas dos países, independentemente do esquema de vacinação. Vamos observar os principais países por:

- Número total de vacinações;
- Percentual de vacinações em relação à população total;
- Número diário de vacinações;
- Número diário de vacinações por milhão de habitantes;
- Pessoas vacinadas;
- Percentual de pessoas vacinadas em relação à população total.

In [116]:
def draw_trace_bar(data, feature, title, xlab, ylab,color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
            x = data['Country'],
            y = data[feature],
            marker=dict(color=color),
            text=data['Country']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=45, 
                           zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                           showline=True, linewidth=2, linecolor='black', mirror=True,
                          tickfont=dict(
                            size=10,
                            color='black'),), 
              yaxis = dict(title = ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                          showline=True, linewidth=2, linecolor='black', mirror=True),
              plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
              hovermode = 'closest'
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')


In [117]:
draw_trace_bar(country_vaccine, 'Total vaccinations', 'Vacinação total por país', 'País', 'Vaccination total', "Darkgreen" )

In [118]:
draw_trace_bar(country_vaccine, 'Percent', 'Porcentagem de vacinação por país', 'País', 'Vaccination percent' )

In [119]:
draw_trace_bar(country_vaccine, 'Daily vaccinations', 'Vacinação diaria por país', 'País', 'Daily vaccinations', "red" )

In [120]:
draw_trace_bar(country_vaccine, 'Daily vaccinations per million', 'Vacinação diária a cada milhão de habitants por país', 'País',\
               'Daily vaccinations per million', "magenta" )

In [121]:
draw_trace_bar(country_vaccine, 'People vaccinated', 'Pessoas vacinadas por país', 'Country',\
               'People vaccinated', "lightblue" )

In [122]:
draw_trace_bar(country_vaccine, 'People vaccinated per hundred', 'Pessoas vacinadas a cada 100 habitantes', 'Country',\
               'People vaccinated per hundred', "orange" )

In [123]:
def plot_custom_scatter(df, x, y, size, color, hover_name, title):
    fig = px.scatter(df, x=x, y=y, size=size, color=color,
               hover_name=hover_name, size_max=80, title = title)
    fig.update_layout({'legend_orientation':'h'})
    fig.update_layout(legend=dict(yanchor="top", y=-0.2))
    fig.update_layout({'legend_title':'Vaccine scheme'})
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='grey')
    fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='grey')
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey')
    fig.show()    

In [124]:
plot_custom_scatter(country_vaccine, x="Total vaccinations", y="Percent", size="Total vaccinations", color="Vaccines",
           hover_name="Country", title = "Vacinações (Percentual vs. total), agrupadas por país e vacinas")

In [125]:
plot_custom_scatter(country_vaccine, x="Total vaccinations", y="Daily vaccinations", size="Total vaccinations", color="Vaccines",
           hover_name="Country", title = "Vacinações (Total vs. Diária) agrupadas por país e vacinas")

In [126]:
plot_custom_scatter(country_vaccine, x="Percent", y="Daily vaccinations per million", size="Total vaccinations", color="Vaccines",
           hover_name="Country", title = "Vacinações (Diária / milhão vs. Percentual) agrupadas por país e vacinas")

In [127]:
# Criação do gráfico de choropleth
trace = go.Choropleth(
    locations = country_vaccine['Country'],  # Locais (nomes dos países)
    locationmode = 'country names',          # Modo de localização baseado em nomes de países
    z = country_vaccine['Total vaccinations'],  # Valores a serem exibidos (vacinações totais)
    text = country_vaccine['Country'],       # Texto a ser exibido ao passar o mouse (nomes dos países)
    autocolorscale = False,                  # Desativar escala de cores automática
    reversescale = True,                     # Inverter a escala de cores
    colorscale = 'viridis',                  # Escala de cores a ser usada
    marker = dict(
        line = dict(
            color = 'rgb(0,0,0)',            # Cor da linha de borda dos países
            width = 0.5                      # Largura da linha de borda dos países
        )
    ),
    colorbar = dict(
        title = 'Total de vacinações',       # Título da barra de cores
        tickprefix = ''                      # Prefixo dos valores da barra de cores
    )
)

# Dados para o gráfico
data = [trace]

# Layout do gráfico
layout = go.Layout(
    title = 'Total de vacinação por paises',  # Título do gráfico
    geo = dict(
        showframe = True,                    # Mostrar a moldura do mapa
        showlakes = False,                   # Não mostrar lagos
        showcoastlines = True,               # Mostrar linhas costeiras
        projection = dict(
            type = 'natural earth'          # Tipo de projeção do mapa
        )
    )
)

# Criação da figura e exibição
fig = dict(data=data, layout=layout)
iplot(fig)


In [128]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['Percent'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Percent',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Total de vacinação a casa 100 habitantes por país.',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [129]:
trace = go.Choropleth(
            locations = country_vaccine['Country'],
            locationmode='country names',
            z = country_vaccine['Daily vaccinations'],
            text = country_vaccine['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'viridis',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Vacinações diárias',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Vacinações diárias por país',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [144]:
trace = go.Choropleth(
    locations = country_vaccine['Country'],
    locationmode = 'country names',
    z = country_vaccine['Daily vaccinations per million'],
    text = country_vaccine['Country'],
    autocolorscale = False,
    reversescale = True,
    colorscale = 'viridis',
    marker = dict(
        line = dict(
            color = 'rgb(0,0,0)',
            width = 0.5
        )
    ),
    colorbar = dict(
        title = 'Vacinações diárias por milhão',
        tickprefix = ''
    )
)

data = [trace]
layout = go.Layout(
    title = 'Vacinações diárias por milhão por país',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict(data = data, layout = layout)
iplot(fig)

In [131]:
trace = go.Choropleth(
    locations = country_vaccine['Country'],
    locationmode = 'country names',
    z = country_vaccine['People vaccinated'],
    text = country_vaccine['Country'],
    autocolorscale = False,
    reversescale = True,
    colorscale = 'viridis',
    marker = dict(
        line = dict(
            color = 'rgb(0,0,0)',
            width = 0.5
        )
    ),
    colorbar = dict(
        title = 'Pessoas vacinadas',
        tickprefix = ''
    )
)

data = [trace]
layout = go.Layout(
    title = 'Pessoas vacinadas por país',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict(data = data, layout = layout)
iplot(fig)


In [132]:
import plotly.graph_objects as go
from plotly.offline import iplot

# Criação do gráfico de choropleth
trace = go.Choropleth(
    locations = country_vaccine['Country'],  # Localizações (nomes dos países)
    locationmode = 'country names',          # Modo de localização baseado em nomes dos países
    z = country_vaccine['People vaccinated'],  # Valores a serem exibidos (pessoas vacinadas)
    text = country_vaccine['Country'],       # Texto a ser exibido ao passar o mouse (nomes dos países)
    autocolorscale = False,                  # Desativar escala de cores automática
    reversescale = True,                     # Inverter a escala de cores
    colorscale = 'viridis',                  # Escala de cores a ser usada
    marker = dict(
        line = dict(
            color = 'rgb(0,0,0)',            # Cor da linha de borda dos países
            width = 0.5                      # Largura da linha de borda dos países
        )
    ),
    colorbar = dict(
        title = 'Pessoas vacinadas',         # Título da barra de cores
        tickprefix = ''                      # Prefixo dos valores da barra de cores
    )
)

# Dados para o gráfico
data = [trace]

# Layout do gráfico
layout = go.Layout(
    title = 'Pessoas vacinadas por país',    # Título do gráfico
    geo = dict(
        showframe = True,                    # Mostrar a moldura do mapa
        showlakes = False,                   # Não mostrar lagos
        showcoastlines = True,               # Mostrar linhas costeiras
        projection = dict(
            type = 'natural earth'          # Tipo de projeção do mapa
        )
    )
)

# Criação da figura e exibição
fig = dict(data = data, layout = layout)
iplot(fig)


<small><a href='#0'>Ir para o início</a></small>  

<a id="5"></a><h1 style='background:#483D8B; border:0; color:white'><center>Como a vacinação está progredindo</center></h1>

Vamos analisar o progresso da vacinação.

Examinaremos os valores de vacinação total e vacinação diária.

In [133]:
country_vaccine_time = data_df[["country", "vaccines", "date", 'total_vaccinations', 
                                'total_vaccinations_per_hundred',  'people_vaccinated','people_vaccinated_per_hundred',
                               'daily_vaccinations','daily_vaccinations_per_million', 
                                'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred'
                               ]].dropna()
country_vaccine_time.columns = ["Country", "Vaccines", "Date", 'Total vaccinations', 'Percent', 'People vaccinated', 'People percent',
                               "Daily vaccinations", "Daily vaccinations per million", 
                                'People fully vaccinated', 'People fully vaccinated percent']

In [134]:
countries = ['Austria', 'Belgium', 'Bulgaria','Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
             'Greece', 'Hungary', 'Ireland', 'Israel', 'Italy', 'Latvia','Lithuania', 'Luxembourg', 'Malta',
             'Netherlands', 'Norway','Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Spain', 'Sweden',
             'United Kingdom', 'United States', 'China']

In [135]:
def plot_time_variation_countries_group(data_df, feature, title, countries):
    data = []
    for country in countries:
        df = data_df.loc[data_df.Country==country]
        trace = go.Scatter(
            x = df['Date'],y = df[feature],
            name=country,
            mode = "markers+lines",
            marker_line_width = 1,
            marker_size = 8,
            marker_symbol = 'circle',
            text=df['Country'])
        data.append(trace)
    layout = dict(title = title,
          xaxis = dict(title = 'Date', showticklabels=True,zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                       showline=True, linewidth=2, linecolor='black', mirror=True,
                       tickfont=dict(size=10,color='darkblue'),), 
          yaxis = dict(title = feature, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                       showline=True, linewidth=2, linecolor='black', mirror=True, type="log"),
                       plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
         hovermode = 'x', 
         height=800
         )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='all_countries')

In [136]:
plot_time_variation_countries_group(country_vaccine_time, 'Percent', 'Evolução percentual da vacinação total (países selecionados, escala logarítmica)', countries)

In [137]:
plot_time_variation_countries_group(country_vaccine_time, 'Total vaccinations', 'Evolução total das vacinações (países selecionados, escala logarítmica)', countries)

In [138]:
plot_time_variation_countries_group(country_vaccine_time, 'People percent', 'Evolução percentual de pessoas vacinadas (países selecionados, escala logarítmica)', countries)

In [139]:
plot_time_variation_countries_group(country_vaccine_time, 'People vaccinated', 'Evolução das pessoas vacinadas (países selecionados, escala logarítmica)', countries)

In [140]:
plot_time_variation_countries_group(country_vaccine_time, 'Daily vaccinations', 'Evolução das vacinações diárias (países selecionados, escala logarítmica)', countries)

In [141]:
plot_time_variation_countries_group(country_vaccine_time, 'Daily vaccinations per million', 'Evolução das vacinações diárias por milhão (países selecionados, escala logarítmica)', countries)

In [142]:
plot_time_variation_countries_group(country_vaccine_time, 'People fully vaccinated percent', 'Evolução da porcentagem de pessoas totalmente vacinadas (países selecionados, escala logarítmica)', countries)

In [143]:
plot_time_variation_countries_group(country_vaccine_time, 'People fully vaccinated', 'Evolução das pessoas totalmente vacinadas (países selecionados, escala logarítmica)', countries)

<small><a href='#0'>Ir para o início</a></small>  